In [7]:
using Printf
using OptimPack.Powell
using LinearAlgebra
using Statistics

-----------------------------------xxx-----------------------------------

BOBYQA ex for R . Maybe help give some idea on how to use
https://rdrr.io/cran/nloptr/man/bobyqa.html

In [ ]:
#Function to minimize
function f23(x::DenseVector{Cdouble})
    return x[1]^2+abs(x[2])
end

In [ ]:
x00=Array{Cdouble}(undef, 2);xl=Array{Cdouble}(undef, 2);
xup=Array{Cdouble}(undef, 2);
    rhobeg = 0.1
    rhoend = 1e-8
x00[1:2].=99;xl[1:2].=-100;xup[1:2].=100

fx = bobyqa!(f23, x00, xl, xup, rhobeg, rhoend, npt=4,verbose=2, maxeval=500000)[1:3]

-----------------------------------xxx-----------------------------------


LC2 matlab code from : http://campar.in.tum.de/Main/LC2Code

In [1]:
#= MATLAB CODE LC2
function [similarity, measure, weight] = LC2Similarity( img1, img2 )
    % img1: one channel image of size n*m
    % img2: multi-channel image of size n*m*c

    similarity =    -1;
    num =   size(img2,1)*size(img2,2);
    ids =   find(img1>0);
    v1 =    var(img1(ids));

    if (v1 > 10^-12)
        if (numel(ids) > num/2)
            img2r =         [reshape(img2, num, size(img2,3)) ones(num,1)];
            pimg2r =        pinv(img2r(ids,:));
            parameter =     pimg2r*img1(ids);
            similarity =    1 - (var(img1(ids) - (img2r(ids,:)*parameter)) / v1);
            weight =        sqrt(v1);
            measure =       weight * similarity;
        end
    end

    if (similarity == -1)
        similarity =    0;
        weight =        0;
        measure =       0;
    end
end
=#

In [98]:
function LC2Similarity(img1,img2)
    similarity =    -1;
    num =   size(img2,1)*size(img2,2);
    ids =   img1 .> 0;
    v1 =    var(img1[ids]);
    
    if (v1 .> 10^-12)
        if (sum(ids) .> num/2)
            img2r =         [reshape(img2, num, size(img2,3)) ones(num,1)];
            pimg2r =        pinv(img2r[vec(ids),:]);
            parameter =     pimg2r*img1[ids];
            similarity =    1 - (var(img1[ids] - (img2r[vec(ids),:]*parameter)) / v1);
            weight =        sqrt(v1);
            measure =       weight * similarity;
        end
    end

    if (similarity == -1)
        similarity =    0;
        weight =        0;
        measure =       0;
    end
    return similarity,weight,measure
    
end

LC2Similarity (generic function with 1 method)

In [99]:
#Testing above code:
img1=randn(10,10);
img2=randn(10,10,10);

In [100]:
(a,b,c)=LC2Similarity(img1,img2)

(0.15995657180069967, 0.4750542777652392, 0.075988053690585)

In [ ]:
#= Patch LC2 code MATLAB
function [similarity, parameter] = LC2SimilarityPatch( img1, img2, patchsize )
    % img1: one channel image of size n*m
    % img2: multi-channel image of size n*m*c
    max_x =         size(img1,1);
    max_y =         size(img1,2);
    totalsize =     ((2*patchsize+1)^2)/2;
    parameter =     zeros(size(img2)+[0 0 1]);
    sim =           zeros(size(img1));
    mea =           zeros(size(img1));
    wei =           zeros(size(img1));

    for y = 1:max_y
        for x = 1:max_x
            patch1 = img1(max(1,x-patchsize):min(max_x,x+patchsize),...
                          max(1,y-patchsize):min(max_y,y+patchsize));
            patch2 = img2(max(1,x-patchsize):min(max_x,x+patchsize),...
                          max(1,y-patchsize):min(max_y,y+patchsize),:);

            if (numel(patch1) > totalsize)
                [sim(x,y), mea(x,y), wei(x,y)] = LC2Similarity( patch1, patch2 );
            end
        end
    end

    similarity = zeros([size(mea,3),1]);

    measure = mea(:,:);
    weights = wei(:,:);
    similarity = sum(measure(:))/sum(weights(:));
    similarity(isnan(similarity)) = 0;
end

=#